## Context Wise MM results

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt

In [ ]:
path_to_preds = Path("../../Preds/MotionSound/")

In [ ]:
# context and activities
contexts = {
    'Bathroom': ['Brushing_hair', 'Hair_dryer_in_use', 'Shaver_in_use', 'Toilet_flushing', 'Toothbrushing', 'Washing_hands'],
    'Kitchen': ['Blender_in_use', 'Chopping', 'Grating', 'Microwave', 'Pouring_pitcher', 'Twisting_jar', 'Washing_Utensils', 'Wiping_with_rag'],
    'Misc': ['Alarm_clock', 'Clapping', 'Coughing', 'Drinking', 'Knocking', 'Laughing', 'Scratching'],
    'Workshop': ['Drill in use', 'Hammering', 'Sanding', 'Screwing', 'Vacuum in use']
}

In [ ]:
sub_sr_metrics = []
for sub_sr_path in path_to_preds.iterdir():
    if sub_sr_path.is_file():
        continue
    sub_sr = sub_sr_path.name
    
    bas = defaultdict(list)
    for fpath in (sub_sr_path).iterdir():
        pred_df = pd.read_csv(fpath)
        pred_df[["PID", "Context", "Activity", "TrialNo"]] = pred_df["file_name"].str.split("---").to_list()
        
        preds = defaultdict(list)
        for name, group in pred_df.groupby(["file_name"]):
            probs = group.drop(columns=["file_name", "y_pred", "y_true", "PID", "Context", "Activity", "TrialNo"])

            group_context = group["Context"].unique()[0]
            group_activity = group["y_true"].unique()[0]
            if group_context == "Other":
                continue
            if group_context == "All":
                continue

            filtered_probs = probs[contexts[group_context]]
            summed_probs = filtered_probs.sum(axis=0)
            y_pred = summed_probs.idxmax()

            preds[group_context].append([group_activity, y_pred])
            
        for context in preds.keys():
            preds_arr = np.array(preds[context])
            bas[context].append(balanced_accuracy_score(preds_arr[:, 0], preds_arr[:, 1]))
    
    for context in bas.keys():
        context_mean = np.mean(bas[context])
        context_std = np.std(bas[context])
        
        sub_sr_metrics.append([sub_sr, context_mean, context_std, context])

In [ ]:
results = pd.DataFrame(sub_sr_metrics, columns=["sub_sr", "mean", "std", "context"])
results["sub_sr"] = results["sub_sr"].astype(int)

In [ ]:
for name, group in results.groupby(["context"]):
    data = group.sort_values(by=["sub_sr"], ascending=False)
    display(data.drop(columns=["context"]).set_index("sub_sr"))
    
    plt.plot(data["sub_sr"], data["mean"])
    plt.ylim([0.4, 1.0])
    plt.xscale("log")
    plt.title(name)
    plt.show()

In [ ]:
# averaged accross contexts
results.groupby("sub_sr").mean().sort_index(ascending=False)

# Context Wise Sound Results

In [ ]:
path_to_preds = Path("../../Preds/Sound/")
sub_sr_metrics = []
for sub_sr_path in path_to_preds.iterdir():
    if sub_sr_path.is_file():
        continue
    sub_sr = sub_sr_path.name
    
    bas = defaultdict(list)
    for fpath in (sub_sr_path).iterdir():
        pred_df = pd.read_csv(fpath)
        pred_df[["PID", "Context", "Activity", "TrialNo"]] = pred_df["file_name"].str.split("---").to_list()
        
        preds = defaultdict(list)
        for name, group in pred_df.groupby(["file_name"]):
            probs = group.drop(columns=["file_name", "y_pred", "y_true", "PID", "Context", "Activity", "TrialNo"])

            group_context = group["Context"].unique()[0]
            group_activity = group["y_true"].unique()[0]
            if group_context == "Other":
                continue
            if group_context == "All":
                continue

            filtered_probs = probs[contexts[group_context]]
            summed_probs = filtered_probs.sum(axis=0)
            y_pred = summed_probs.idxmax()

            preds[group_context].append([group_activity, y_pred])
            
        for context in preds.keys():
            preds_arr = np.array(preds[context])
            bas[context].append(balanced_accuracy_score(preds_arr[:, 0], preds_arr[:, 1]))
    
    for context in bas.keys():
        context_mean = np.mean(bas[context])
        context_std = np.std(bas[context])
        
        sub_sr_metrics.append([sub_sr, context_mean, context_std, context])
        
results = pd.DataFrame(sub_sr_metrics, columns=["sub_sr", "mean", "std", "context"])
results["sub_sr"] = results["sub_sr"].astype(int)

In [ ]:
for name, group in results.groupby(["context"]):
    data = group.sort_values(by=["sub_sr"], ascending=False)
    display(data.drop(columns=["context"]).set_index("sub_sr"))
    
    plt.plot(data["sub_sr"], data["mean"])
    plt.ylim([0.4, 1.0])
    plt.xscale("log")
    plt.title(name)
    plt.show()

In [ ]:
results.groupby("sub_sr").mean().sort_index(ascending=False)

In [ ]:
path_to_preds = Path("../../Preds/Motion/")

bas = defaultdict(list)
for fpath in (path_to_preds).iterdir():
    pred_df = pd.read_csv(fpath)
    pred_df[["PID", "Context", "Activity", "TrialNo"]] = pred_df["file_name"].str.split("---").to_list()

    preds = defaultdict(list)
    for name, group in pred_df.groupby(["file_name"]):
        probs = group.drop(columns=["file_name", "y_pred", "y_true", "PID", "Context", "Activity", "TrialNo"])

        group_context = group["Context"].unique()[0]
        group_activity = group["y_true"].unique()[0]
        if group_context == "Other":
            continue
        if group_context == "All":
            continue

        filtered_probs = probs[contexts[group_context]]
        summed_probs = filtered_probs.sum(axis=0)
        y_pred = summed_probs.idxmax()

        preds[group_context].append([group_activity, y_pred])

    for context in preds.keys():
        preds_arr = np.array(preds[context])
        bas[context].append(balanced_accuracy_score(preds_arr[:, 0], preds_arr[:, 1]))

results = []
for context in bas.keys():
    context_mean = np.mean(bas[context])
    context_std = np.std(bas[context])

    results.append([context, context_mean, context_std])
    # sub_sr_metrics.append([sub_sr, context_mean, context_std, context])
results = pd.DataFrame(results, columns=["context", "mean", "std"])

In [ ]:
results